In [2]:
##Imports
import pandas as pd
import pandas_datareader.data as web
from pandas import DataFrame, Series
import datetime
from datetime import timedelta, date

In [4]:
stock = "F"
start = (date.today() - timedelta(weeks=260))
end = date.today()
stockDf = web.DataReader(stock, 'yahoo', start, end)
stockDf

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-12-12,13.100000,12.810000,13.080000,12.820000,39596800.0,10.391035
2016-12-13,12.880000,12.750000,12.840000,12.770000,39570500.0,10.350507
2016-12-14,12.790000,12.500000,12.650000,12.530000,42820100.0,10.155979
2016-12-15,12.720000,12.520000,12.540000,12.580000,23774800.0,10.196507
2016-12-16,12.700000,12.560000,12.660000,12.630000,28354900.0,10.237033
...,...,...,...,...,...,...
2021-11-29,20.190001,19.490000,20.070000,19.670000,68778700.0,19.670000
2021-11-30,19.950001,19.030001,19.620001,19.190001,103238400.0,19.190001
2021-12-01,20.469999,19.530001,19.629999,19.580000,125295600.0,19.580000


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-12-16,69.190001,68.140001,68.770000,68.330000,174117105.0,56.331060
2016-12-23,68.490001,67.670000,68.080000,67.940000,102223405.0,56.014954
2016-12-30,53.550000,52.880000,53.290000,53.000000,93567604.0,43.716122
2017-01-06,55.930001,54.140000,54.980000,55.290001,234093204.0,45.572248
2017-01-13,68.950001,67.650001,68.480000,68.370001,213551005.0,56.363484
...,...,...,...,...,...,...
2021-11-05,99.209999,95.389997,97.199999,98.300001,624621605.0,97.832567
2021-11-12,105.860001,101.179998,103.809998,103.680000,721665405.0,103.185612
2021-11-19,104.950001,101.710001,103.899998,103.559999,408230205.0,103.261353
